### Pre-requisites
1) Azure Subscription

2) Create Azure Machine Learning Service resource https://learn.microsoft.com/en-us/azure/machine-learning/quickstart-create-resources
or using Azure Portal https://ms.portal.azure.com/#create/Microsoft.MachineLearningServices

3) You can spin -up a compute instance in the Azure ML workspace and use this as development environment or you can use your own development environment (your desktop/laptop)

4) Install the SDK v2

_**pip install azure-ai-ml**_

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Enter details of your AML workspace
subscription_id = "<subscription-id>"
resource_group = "<resource-group>"
workspace = "<workspace-name>"

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

### Register your model to workspace ###

In [1]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path= "./model/sklearn_regression_model.pkl",#path to your model file
    type=AssetTypes.CUSTOM_MODEL,
    name="sklearnmodel-localfile",#provide model name of your choice
    description="Model created from local file",#if you would like to add a description
)
ml_client.models.create_or_update(file_model)

### Creating Online Endpoint

In [6]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
import datetime

# Creating a unique endpoint name 
online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f") #you may create a unique endpoint name of your choice

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="sklearnmodel-endpoint",
    auth_mode="key",
    tags={"outcome": "sklearnprediction"} ## Just an example. You can provide your own tags. 
)

ml_client.begin_create_or_update(endpoint)

### Deploy Model, Environment and Scoring Script to the Endpoint created.

In [26]:
### Environment for deployment
env = Environment(
    conda_file="./environment/conda.yml", # you cn create ,yml file with your own dependencies.
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1", # This is the base image. 
)


blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model="sklearnmodel-localfile@latest", ## model name registered along with version
    environment=env,
    code_configuration=CodeConfiguration(
        code="./onlinescoring", scoring_script="score.py" # point to the location where scoring script is stored and pass the name of the scoring script.
    ),
    instance_type="Standard_F2s_v2", ### VM size to be used.
    instance_count=1, ### As of now we will start with 1 and then we can increase.
)

ml_client.begin_create_or_update(blue_deployment)